In [7]:
import psycopg2
from pymongo import MongoClient
from bson import ObjectId
import pandas as pd

In [8]:
# !pip install psycopg2-binary pymongo pandas

In [10]:
# Connexion PostgreSQL
pg_conn = psycopg2.connect(
    dbname="projet_bdd",
    user="postgres",         # 🔁 adapte ton utilisateur
    password="ssh",   # 🔁 adapte ton mot de passe
    host="localhost",
    port="5432"
)
pg_cursor = pg_conn.cursor()

In [11]:
# Connexion MongoDB
mongo_client = MongoClient("mongodb://localhost:27017/")
mongo_db = mongo_client["projet_bdd"]
commentaires_collection = mongo_db["commentaires"]

In [15]:
# Fonction : récupérer les avis d’un hébergement avec commentaires enrichis
def avis_avec_commentaires(hebergement_id):
    # Étape 1 : récupérer les avis depuis PostgreSQL
    pg_cursor.execute("""
        SELECT id, note, utilisateur, date_avis, commentaire_id
        FROM Avis
        WHERE hebergement_id = %s
    """, (hebergement_id,))
    avis_sql = pg_cursor.fetchall()

    # Étape 2 : récupérer les commentaires depuis MongoDB
    commentaire_ids = [ObjectId(row[4]) for row in avis_sql]
    commentaires = {
        str(doc["_id"]): doc for doc in commentaires_collection.find(
            {"_id": {"$in": commentaire_ids}}
        )
    }

    # Étape 3 : fusionner les données
    data = []
    for row in avis_sql:
        id_, note, user, date, commentaire_id = row
        commentaire_doc = commentaires.get(commentaire_id, {})
        data.append({
            "Utilisateur": user,
            "Note": note,
            "Date": date,
            "Commentaire": commentaire_doc.get("contenu", "N/A"),
            "Images": commentaire_doc.get("images", []),
        })

    return pd.DataFrame(data)

In [17]:
#  Exemple d'appel
avis_avec_commentaires(1)


,Utilisateur,Note,Date,Commentaire,Images
0,alice@example.com,5,2025-05-20,"Très bon séjour, personnel accueillant.",[]
1,bob@example.com,4,2025-05-22,Bon rapport qualité prix.,[]


In [18]:
from datetime import datetime
from bson import ObjectId

def ajouter_avis_et_commentaire(hebergement_id, utilisateur, note, contenu, images=None, reponses=None):
    # Valeurs par défaut
    images = images or []
    reponses = reponses or []

    # Création du document commentaire
    commentaire = {
        "contenu": contenu,
        "images": images,
        "date_commentaire": datetime.utcnow(),
        "reponses": reponses  # liste de dictionnaires {"utilisateur":..., "contenu":..., "date":...}
    }

    # Insertion dans MongoDB
    result = commentaires_collection.insert_one(commentaire)
    commentaire_id = str(result.inserted_id)

    # Insertion de l'avis lié dans PostgreSQL
    pg_cursor.execute("""
        INSERT INTO Avis (note, utilisateur, date_avis, commentaire_id, hebergement_id)
        VALUES (%s, %s, %s, %s, %s)
    """, (
        note,
        utilisateur,
        datetime.today().date(),
        commentaire_id,
        hebergement_id
    ))
    pg_conn.commit()

    print(f"✅ Avis et commentaire insérés. ID du commentaire : {commentaire_id}")


In [19]:
ajouter_avis_et_commentaire(
    hebergement_id=2,
    utilisateur="lea@example.com",
    note=4,
    contenu="Très belle auberge, je recommande.",
    images=["photo1.jpg", "photo2.jpg"],
    reponses=[
        {
            "utilisateur": "auberge@example.com",
            "contenu": "Merci pour votre retour 😊",
            "date": datetime(2025, 5, 30)
        }
    ]
)


✅ Avis et commentaire insérés. ID du commentaire : 684f7d7a29bc3be0dfbd8ff2


In [ ]:
684f7d7a29bc3be0dfbd8ff2
684f7d7a29bc3be0dfbd8ff2

In [20]:
def ajouter_avis_et_commentaire(conn_pg, collection_mongo, note, utilisateur, date_avis, hebergement_id, **kwargs):
    from datetime import datetime
    from bson import ObjectId
    import psycopg2.extras

    # 1. Créer le commentaire MongoDB avec champs dynamiques
    commentaire = {
        "contenu": kwargs.get("contenu", ""),
        "date_commentaire": kwargs.get("date_commentaire", datetime.utcnow()),
        "avis_id": kwargs.get("avis_id"),  # peut être mis à jour ensuite
        "reponses": kwargs.get("reponses", [])
    }

    # Ajouter les autres champs dynamiques comme vidéos, liens...
    for key in kwargs:
        if key not in commentaire:
            commentaire[key] = kwargs[key]

    result = collection_mongo.insert_one(commentaire)
    commentaire_id = str(result.inserted_id)

    # 2. Insérer l'avis dans PostgreSQL
    cur = conn_pg.cursor()
    cur.execute("""
        INSERT INTO Avis (note, utilisateur, date_avis, commentaire_id, hebergement_id)
        VALUES (%s, %s, %s, %s, %s)
        RETURNING id
    """, (note, utilisateur, date_avis, commentaire_id, hebergement_id))
    conn_pg.commit()
    cur.close()
    return commentaire_id


In [22]:
ajouter_avis_et_commentaire(
    pg_conn, commentaires_collection,
    note=4,
    utilisateur="nora@example.com",
    date_avis="2025-06-15",
    hebergement_id=2,
    contenu="Expérience magique !",
    videos=["https://youtu.be/video123"],
    tags=["été", "romantique"]
)


'684f804029bc3be0dfbd8ff3'

In [2]:
from neo4j import GraphDatabase
import psycopg2

# 🔗 Connexion PostgreSQL
conn_pg = psycopg2.connect(
    dbname="projet_bdd",
    user="postgres",
    password="ssh",
    host="localhost",
    port=5432
)
cursor_pg = conn_pg.cursor()

# 🔗 Connexion Neo4j
driver_neo4j = GraphDatabase.driver("bolt://localhost:7687", auth=("neo4j", "projet_bdd"))

# 🚀 Fonction pour insérer une ville dans les deux bases
def ajouter_ville(nom, latitude, longitude):
    # PostgreSQL
    cursor_pg.execute("""
        INSERT INTO Ville (nom, latitude, longitude)
        VALUES (%s, %s, %s)
        ON CONFLICT (nom) DO NOTHING;
    """, (nom, latitude, longitude))
    conn_pg.commit()

    # Neo4j
    with driver_neo4j.session() as session:
        session.run("""
            MERGE (:Ville {nom: $nom})
        """, nom=nom)

# Exemple d’appel
villes_a_ajouter = [
    ("Longueuil", 45.5312, -73.5181),
    ("Laval", 45.6066, -73.7124),
    ("Gatineau", 45.4765, -75.7013),
    ("Mississauga", 43.5890, -79.6441),
    ("Burnaby", 49.2488, -122.9805),
    ("Dartmouth", 44.6658, -63.5671)
]

for ville in villes_a_ajouter:
    ajouter_ville(*ville)

# 🔒 Fermeture
cursor_pg.close()
conn_pg.close()
driver_neo4j.close()


In [6]:
!!pip install neo4j --upgrade

['Requirement already satisfied: neo4j in c:\\users\\mlds\\anaconda3\\lib\\site-packages (5.28.1)',
 'Requirement already satisfied: pytz in c:\\users\\mlds\\anaconda3\\lib\\site-packages (from neo4j) (2021.3)']